In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('Ressources/Data/Camp_Market.csv', sep=',')

In [3]:
print(pd.get_option("display.max_rows"))
print(pd.get_option("display.max_columns"))

60
20


In [4]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce').dt.strftime('%d/%m/%Y')

/var/folders/4n/0hqdswrx4pn48s5909d75wzr0000gn/T/ipykernel_22798/1408691525.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce').dt.strftime('%d/%m/%Y')


In [5]:
cols_to_check = [
    'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
    'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'
]

duplicate_mask = df.duplicated(subset=cols_to_check, keep='first')

df = df.drop_duplicates(subset=cols_to_check, keep='first').reset_index(drop=True)
df.to_csv("Ressources/Data/Camp_Market_Cleaned.csv", index=False)

In [6]:
df['Marital_Status'] = df['Marital_Status'].replace({
    'Absurd': 'Others',
    'YOLO': 'Others',
    'Alone': 'Single'
})

df['Education'] = df['Education'].replace({
    'Basic': 'Highschool',
})

In [7]:
missing_idx = df['Income'].isna()  

def impute_income(row, df):
    if pd.notna(row['Income']):
        return row['Income'] 
    
    mask = (
        (df['Year_Birth'] == row['Year_Birth']) &
        (df['Education'] == row['Education']) &
        (df['Marital_Status'] == row['Marital_Status']) &
        (df['Income'].notna())
    )
    
    group_values = df.loc[mask, 'Income']
    
    if len(group_values) > 0:
        return round(group_values.mean()) 
    else:
        return np.nan

df['Income'] = df.apply(lambda row: impute_income(row, df), axis=1).astype('Int64')


In [8]:
display(df[missing_idx & df['Income'].notna()])

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
10,1994,1983,Graduation,Married,46372,1,0,15/11/2013,11,5,...,7,0,0,0,0,0,0,3,11,0
27,5255,1986,Graduation,Single,30846,1,0,20/02/2013,19,5,...,1,0,0,0,0,0,0,3,11,0
43,7281,1959,PhD,Single,34554,0,0,05/11/2013,80,81,...,2,0,0,0,0,0,0,3,11,0
48,7244,1951,Graduation,Single,55593,2,1,01/01/2014,96,48,...,6,0,0,0,0,0,0,3,11,0
58,8557,1982,Graduation,Single,45618,1,0,17/06/2013,57,11,...,6,0,0,0,0,0,0,3,11,0
71,10629,1973,2n Cycle,Married,37098,1,0,14/09/2012,25,25,...,8,0,0,0,0,0,0,3,11,0
89,8996,1957,PhD,Married,59772,2,1,19/11/2012,4,230,...,9,0,0,0,0,0,0,3,11,0
90,9235,1957,Graduation,Single,57748,1,1,27/05/2014,45,7,...,7,0,0,0,0,0,0,3,11,0
127,8268,1961,PhD,Married,55764,0,1,11/07/2013,23,352,...,6,0,0,0,0,0,0,3,11,0
132,1295,1963,Graduation,Married,57604,0,1,11/08/2013,96,231,...,4,0,0,0,0,0,0,3,11,0


In [9]:
df['Nmb_Childrens'] = df['Kidhome'] + df['Teenhome']
df['Total_Mnt'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

In [10]:
df.to_csv("Ressources/Data/Camp_Market_Cleaned.csv", index=False)

2902	1958	Graduation	Together		1	1	9/3/12	87	19	4	12	2	2	6	1	1	0	3	5	0	0	0	0	0	0	3	11	0
7281	1959	PhD	Single		0	0	11/5/13	80	81	11	50	3	2	39	1	1	3	4	2	0	0	0	0	0	0	3	11	0